In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import keras_tuner
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the data
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")

In [ ]:
# Take out the sample
sample_train = train.sample(frac = 0.1, random_state=100)
sample_validation = train.sample(frac = 0.05, random_state=101)
sample_test = train.sample(frac = 0.05, random_state=102)

In [ ]:
# Split the data into features and target variable

X_train = sample_train.drop(['id', 'target'], axis=1)
y_train = sample_train['target']

X_val = sample_validation.drop(['id', 'target'], axis=1)
y_val = sample_validation['target']

X_test = sample_test.drop(['id', 'target'], axis=1)
y_test = sample_test['target']

# Scale the data

mms = MinMaxScaler()
X_train_scaled = mms.fit_transform(X_train)
X_val_scaled = mms.transform(X_val)
X_test_scaled = mms.transform(X_test)

In [ ]:
# Setting up metrics and callbacks
m = keras.metrics.AUC(curve='ROC')
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# Using KerasTuner to chose hyperparameters
def build_model(hp):
    model = keras.Sequential()
    # Adding 2 to 20 layers
    for i in range(hp.Int('num_layers',2, 20)):
        
        # add hidden layers and neurons
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                           min_value=32, 
                                           max_value=512,
                                           step=32),
                              activation='relu'))

    # Add the output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-1, 1e-2, 1e-3, 1e-4])),
            loss='binary_crossentropy',
            metrics = [m])

    return model

In [ ]:
tuner = RandomSearch(
        build_model,
        objective=keras_tuner.Objective("val_auc", direction="max"),
        max_trials=5, 
        executions_per_trial=3,
        directory='MyProject',
        project_name='TPS_DL')

In [ ]:
# Print the summary
tuner.search_space_summary()

In [ ]:
# Perform the search operation
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_val_scaled, y_val), callbacks = [stop_early])

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_model.evaluate(X_train_scaled, y_train)

In [ ]:
best_model.evaluate(X_val_scaled, y_val)

In [ ]:
best_model.evaluate(X_test_scaled, y_test)

# Train the model on the entire data

In [ ]:
# Transform the data
final_training = mms.transform(train.drop(['id', 'target'], axis=1))
target_var = train['target']

# fit the model
best_model.fit(final_training, target_var, epochs=20)

# Make Submissions

In [ ]:
test_scaled = mms.transform(test.drop(['id'], axis=1))

In [ ]:
predictions = np.round(best_model.predict(test_scaled)).astype('int')
ids = test.id

In [ ]:
# Store the submissions in the dataframe
submissions = pd.DataFrame(np.concatenate((ids.values.reshape(-1,1), predictions), axis=1), columns=['id', 'target'])

In [ ]:
# Plot the predictions
figure, ax = plt.subplots(1, 2, figsize=(12,8))
sns.countplot(data=submissions, x='target', ax = ax[0])
ax[0].set_title("Predictions")

sns.countplot(data=train, x='target', ax = ax[1])
ax[1].set_title("Actual")

figure.show()

In [ ]:
# Make Submissions
submissions.to_csv('././MyProject/TPS_DL/dl_submissions.csv', index=False)